# Data access and analysis ModBase

MODBASE is a queryable database of annotated protein structure models. The models are derived by ModPipe, an automated modeling pipeline relying on the programs PSI-BLAST and MODELLER. The database also includes the fold assignments and alignments on which the models were based. MODBASE contains theoretically calculated models, which may contain significant errors.[Reference](https://salilab.org/pdf/Pieper_NucleicAcidsRes_2013a.pdf)

Data Source: [ModBase](https://salilab.org/modbase-download/projects/genomes/H_sapiens/2020/)
The sequences in the Homo_sapiens_2020 dataset come from both **RefSeq** and **Ensembl**.
Models are provided in PDB format. For mmCIF files, use the modbase_pdb_to_cif.py script from the modbase_utils GitHub repository to generate them. ModBase mmCIF files are compliant with the PDBx and MA mmCIF dictionaries.


In [2]:
#file = 'Homo_sapiens_2020/models'
import requests
import pandas as pd
from bs4 import BeautifulSoup
url = "https://salilab.org/modbase-download/projects/genomes/H_sapiens/2020/Homo_sapiens_2020.summary.txt"
df = pd.read_csv(url, sep='\t')
df.head()

/Users/harmeet/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,run_name,seq_id,model_id,database_id,target_beg,target_end,sequence identity,evalue,ga341,mpqs,zdope,pdb_code,pdb_chain,pdb_beg,pdb_end,hit history,tsvmod_method,tsvmod_no35,tsvmod_rmsd
0,MW-Human_D15,7c658f4efe7d85744ca465a0267f7792MGLTLSKR,edc277ca01ade5f0777b50c53d8ab118,ENSP00000000233.5_1,2,180,80,0.0,1.0,2.05954,-1.84,1r8q,A,2,180,1,MTALL,0.897,3.280
1,MW-Human_D15,7c658f4efe7d85744ca465a0267f7792MGLTLSKR,fda3df48c81a6da87145981a37d91191,ENSP00000000233.5_2,7,180,99,0.0,1.0,2.27067,-2.42,2b6h,A,7,180,1000,MTALL,0.860,2.076
2,MW-Human_D15,7c658f4efe7d85744ca465a0267f7792MGLTLSKR,cc139a2c712c06e1079d63bc52dc4c1d,ENSP00000000233.5_3,58,177,60,0.0,1.0,1.51077,-1.63,3lvq,E,54,173,1,MSALL,0.944,1.502
3,MW-Human_D15,a55542552511584af7ba8abd011710dbMFPFLLPM,37aa8907205617905b70e266fd19c6de,ENSP00000000412.3_1,29,180,89,0.0,1.0,1.61874,-1.57,3k43,A,3,154,1000,MSALL,0.954,0.551
4,MW-Human_D15,cf7275554bf5d843811badb432d949acMSSQAMMD,215b59d7a0dbe44a3b1aff19b8ffe2fb,ENSP00000000442.6_1,72,419,36,0.0,1.0,1.15560,0.10,4nqa,A,128,456,1000,MSALL,0.787,5.135



<img src="pipeline.png">

In [3]:
#Check basic info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 475451 entries, 0 to 475450
Data columns (total 19 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   run_name           475451 non-null  object 
 1   seq_id             475451 non-null  object 
 2   model_id           475451 non-null  object 
 3   database_id        475451 non-null  object 
 4   target_beg         475451 non-null  int64  
 5   target_end         475451 non-null  int64  
 6   sequence identity  475451 non-null  int64  
 7   evalue             475451 non-null  float64
 8   ga341              475451 non-null  float64
 9   mpqs               475451 non-null  float64
 10  zdope              475451 non-null  float64
 11  pdb_code           475451 non-null  object 
 12  pdb_chain          475451 non-null  object 
 13  pdb_beg            475451 non-null  object 
 14  pdb_end            475451 non-null  object 
 15  hit history        475451 non-null  int64  
 16  ts

* **ModBase seq_id** (unique for a given sequence), 
* **ModBase model_id** (unique for a given model), 
* **database_id** (the primary identifier from RefSeq or Ensembl for the sequence for which the model was built, with a suffix since multiple models may exist for a single sequence).

Multiple database IDs may exist for a given model_id or seq_id (for example, the sequence may be in both RefSeq and Ensembl). Only one of the IDs is listed in the summary table. 


### Quality criteria:
For each selected target–template alignment, 10 models are
calculated, and the model with the best value of the
DOPE statistical potential is selected and then
evaluated by several additional quality criteria: (i)
target–template sequence identity, (ii) **GA341** score,
(iii) **Z-DOPE** score, (iv) **MPQS** score (ModPipe
quality score) (v) **TSVMod** score. 

* **MPQS (ModPipe Quality Score): >=1.1
* **TSVMod NO35 (estimated native overlap at 3.5 Å): >=40%
* **GA341: >=0.7
* **E-value: <0.0001
* **zDOPE: <0
* **Sequence Identity**

**Z-DOPE** with value equal to -1 or less can be considered best models.
n-DOPE is a z-score. So, the lower the n-DOPE the better.
Using the probability theory, we derive an atomic distance- dependent statistical potential from a sample of native structures that does not depend on any adjustable parameters (Discrete Optimized Protein Energy, or DOPE). DOPE is based on an improved reference state that corresponds to noninteracting atoms in a homogeneous sphere with the radius dependent on a sample native structure; it thus accounts for the finite and spherical shape of the native structures

**E-Value**
ModPipe1.0: Significance of the alignment between the target and the template as reported by NCBI's PSI-BLAST program (Nucl. Acids Res. 25, 3389-3402, 1997). This is the significance reported during the template (PDB) database search. It is not the significance of the modeling alignment produced by Modeller. ModPipe2 and later:Similar significance value, but calculated by Modeller using the Build-Profile routine.

**GA341 (Model Score)** ranges from 0 to 1.
Assesses the quality of the model by assessing fold assignment. The method uses the percentage sequence identity between the template and the model as a parameter. 
Score for the reliability of a Model, derived from statistical potentials (F. Melo, R. Sanchez, A. Sali,2002 PDF). A model is predicted to be **good when the model score is higher than a pre-specified cutoff (0.7)**. A reliable model has a probability of the correct fold that is larger than 95%. A fold is correct when at least 30% of its Cα atoms superpose within 3.5 Å of their correct positions.[Reference](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC2211691/)

Protein Size
Length of the modeled sequences (original sequence, not the modeled part).

Model Size
Length of the model;

Reliable Model / Fold Assignment
A model is considered to be reliable (have a reliable fold assignment) if it is evaluated within the following thresholds by at least one of theses model evaluation criteria:

ModPipe Protein Quality Score
The ModPipe Protein Quality Score is a composite score comprising sequence identity to the template, coverage, and the three individual scores evalue, z-Dope and GA341. We consider a MPQS of >1.1 as reliable.

**TSVMod**
* MTALL: Training set is based on the template structure (most reliable method)
* MSALL: Training set is based on similar secondary structure, all features are used
* MSRED: For lack of sufficiently similar secondary structures in the training set, a reduced set of z-scores features is used to compute the results.
* NA: No prediction, due to secondary structure that is entirely missing from the training set.
* NO35: Predicted Native Overlap (3.5 Å)
* RMSD: Predicted RMSD**
Reference: D. Eramian, N. Eswar, M.Y. Shen, A. Sali. How well can the accuracy of comparative protein structure models be predicted? Protein Sci 17, 1881-1893, 2008.



In [13]:
# Check unique sequence ids and their counts
df['seq_id'].value_counts()

144577fc51b1de946e72180a9e41dae7MTTQIRSI    232
a2b1384bcd9cb341e98b078cfedb91b7MTTQIRSI    231
b5f8e696790ead21221ee0f08a03e944MTTQIRSI    221
14a9519676941ba00237e7fe90ceada1MSSQIRSI    219
c3179a9053f2014e5d87e095520fcb7dMTTQIRSI    217
                                           ... 
f76710e98db89c8dd87e37f111fd0d15MASGEEVE      1
87de1546c7ac08585cbe676bd49a951eMIRFQSLE      1
9800a97afa7a503160a6d594234e9278MKGTFTIQ      1
b0fd1fde08548b44e4071849cc551f6eMAALSCCQ      1
b6379853084fdf76c9bb53cdceae0aa5MEQLESSK      1
Name: seq_id, Length: 153964, dtype: int64

Unique sequences covered is 153964

In [16]:
# Check unique database ids and their counts
df['database_id'].value_counts()

XP_005257157.2_4        1
ENSP00000347444.4_4     1
ENSP00000372335.4_2     1
ENSP00000419092.1_1     1
ENSP00000439910.2_3     1
                       ..
ENSP00000478842.1_2     1
ENSP00000394405.3_1     1
XP_016880000.1_1        1
ENSP00000251170.6_5     1
GENSCAN00000007656_1    1
Name: database_id, Length: 475451, dtype: int64

## Check one case with multiple models

In [17]:
# Check a single seq_id
seq1= df.loc[df['seq_id']== '144577fc51b1de946e72180a9e41dae7MTTQIRSI']
seq1

,run_name,seq_id,model_id,database_id,target_beg,target_end,sequence identity,evalue,ga341,mpqs,zdope,pdb_code,pdb_chain,pdb_beg,pdb_end,hit history,tsvmod_method,tsvmod_no35,tsvmod_rmsd
473016,MW-Human_D15,144577fc51b1de946e72180a9e41dae7MTTQIRSI,65b72737a2b79a5621779260092a2a07,XP_024308866.1_1,1,194,100,0.00000,1.00,1.103870,-0.80,2a38,A,1,194,1000,MTALL,0.894,2.453
473017,MW-Human_D15,144577fc51b1de946e72180a9e41dae7MTTQIRSI,74c5f9913ab81806ea21db09f4761d37,XP_024308866.1_2,103,243,76,0.00000,1.00,0.746268,0.06,6fwx,B,103,238,1000,MSALL,0.930,2.622
473018,MW-Human_D15,144577fc51b1de946e72180a9e41dae7MTTQIRSI,e5a56224a97be4ef9c9e9910935f81ba,XP_024308866.1_3,940,1035,31,0.00000,0.98,0.391806,-1.34,1fhg,A,38,134,1,MTALL,0.939,1.676
473019,MW-Human_D15,144577fc51b1de946e72180a9e41dae7MTTQIRSI,0c36508043f81a0f3e8e59f6405e243a,XP_024308866.1_4,941,1034,28,0.00000,1.00,0.326745,-0.98,1g1c,A,3,97,1,MTALL,0.898,1.734
473020,MW-Human_D15,144577fc51b1de946e72180a9e41dae7MTTQIRSI,370db54069845bebe940dd110c7bc729,XP_024308866.1_5,944,1033,39,0.00015,1.00,0.443724,-0.63,2yr3,A,12,98,1000,MTALL,0.877,2.852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
473243,MW-Human_D15,144577fc51b1de946e72180a9e41dae7MTTQIRSI,e25a6a8f02f443e60d214fec4c952efa,XP_024308866.1_228,31404,31500,100,0.00000,1.00,1.146940,-1.26,6hci,A,8,104,1000,MSALL,1.000,0.541
473244,MW-Human_D15,144577fc51b1de946e72180a9e41dae7MTTQIRSI,4b90d587ac0e8ffc448f8a06b79e5280,XP_024308866.1_229,31981,32076,100,0.00000,1.00,1.173910,-1.53,6h4l,A,4,99,1000,MSALL,1.000,0.184
473245,MW-Human_D15,144577fc51b1de946e72180a9e41dae7MTTQIRSI,7c9177bd5547b662785cf77ad06bb884,XP_024308866.1_230,32459,32555,100,0.00000,1.00,1.166940,-1.46,3puc,A,2,98,1000,MSALL,1.000,0.169
473246,MW-Human_D15,144577fc51b1de946e72180a9e41dae7MTTQIRSI,78d73fc5ca0a171c6ad96ca9b4662eb4,XP_024308866.1_231,32654,32837,18,0.02100,1.00,-0.116830,0.48,5dhx,B,5,226,200,MSALL,0.233,6.725


231 models for a sequence

[Protein Sequence](https://www.ncbi.nlm.nih.gov/protein/XP_024308866.1)
* titin isoform X9 
* Sequence length : 33035


[Check model details](https://modbase.compbio.ucsf.edu/modbase-cgi/model_details.cgi?queryfile=1618582421_1114&searchmode=default&displaymode=moddetail&seq_id=144577fc51b1de946e72180a9e41dae7MTTQIRSI)

#### Check number of templates used

In [20]:
seq1['pdb_code'].value_counts()

5j7k    5
2e7c    3
1uem    3
3dmk    3
4u3h    3
       ..
3lcy    1
2dn7    1
4hwu    1
5ope    1
3ry4    1
Name: pdb_code, Length: 189, dtype: int64

# Summary/Observations:
    
    - The complete set of models and alignment files for homo sapiens can be found in the development platform.
    - Models are in pdb format and can be converted to mmCIF format using 'modbase_pdb_to_cif.py' script from the modbase_utils GitHub repository
    - Gaps are modeled as loops with coordinates in pdb file
    - Models are in different frames
    